In [ ]:
!pip install --force-reinstall numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
captum 0.8.0 requires numpy<2.0, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.


In [ ]:
!pip install transformers transformers-interpret torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary modules
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer
import torch
import numpy as np

# Load model and tokenizer
model_path = "/content/drive/MyDrive/new_model/"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
model.eval()

# Create explainer
explainer = SequenceClassificationExplainer(model, tokenizer)

# Sample text input
text = "Metformin is used to help control diabetes, but it does not cure it."
attributions = explainer(text)

# Merge subwords and aggregate scores
merged_tokens = []
merged_scores = []
current_token = ""
current_score = 0.0

for token, score in attributions:
    if token.startswith("##"):
        current_token += token[2:]
        current_score += score
    else:
        if current_token:
            merged_tokens.append(current_token)
            merged_scores.append(current_score)
        current_token = token
        current_score = score

# Add last token
if current_token:
    merged_tokens.append(current_token)
    merged_scores.append(current_score)

# Filter to positive scores
positive_scores = [s for s in merged_scores if s > 0]
mean_score = np.mean(positive_scores) if positive_scores else 0
std_score = np.std(positive_scores) if positive_scores else 0
threshold = mean_score + 0.5 * std_score

# Get tokens above threshold
important_tokens = [
    (token, score) for token, score in zip(merged_tokens, merged_scores) if score > threshold
]

# Fallback: If no tokens pass, show top 2 tokens with highest positive scores
if not important_tokens:
    sorted_tokens = sorted(
        [(token, score) for token, score in zip(merged_tokens, merged_scores) if score > 0],
        key=lambda x: x[1],
        reverse=True
    )
    important_tokens = sorted_tokens[:2]

# Display results
print("Text:", text)
print("Important Tokens:", important_tokens)
print("Threshold:", round(threshold, 4))


📝 Text: Metformin is used to help control diabetes, but it does not cure it.
📌 Important Tokens: [('control', 0.051416024565696716), ('diabetes', 0.07729330658912659)]
📊 Threshold: 0.0422


In [ ]:
# Import necessary modules
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer
import torch
import numpy as np

# Load model and tokenizer
model_path = "/content/drive/MyDrive/new_model/"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
model.eval()

# Create explainer
explainer = SequenceClassificationExplainer(model, tokenizer)

# List of 5 test sentences
test_sentences = [
    "Insulin can reverse diabetes permanently.",
    "Drinking bitter leaf tea cures diabetes overnight.",
    "Regular exercise helps manage blood sugar levels effectively.",
    "Using herbal supplements alone can replace diabetes medication.",
    "Metformin improves insulin sensitivity but is not a permanent solution."
]

# Function to process and extract important tokens
def get_important_tokens(text):
    attributions = explainer(text)

    # Merge subwords
    merged_tokens = []
    merged_scores = []
    current_token = ""
    current_score = 0.0

    for token, score in attributions:
        if token.startswith("##"):
            current_token += token[2:]
            current_score += score
        else:
            if current_token:
                merged_tokens.append(current_token)
                merged_scores.append(current_score)
            current_token = token
            current_score = score

    if current_token:
        merged_tokens.append(current_token)
        merged_scores.append(current_score)

    # Filter to positive scores
    positive_scores = [s for s in merged_scores if s > 0]
    mean_score = np.mean(positive_scores) if positive_scores else 0
    std_score = np.std(positive_scores) if positive_scores else 0
    threshold = mean_score + 0.5 * std_score

    # Get tokens above threshold
    important_tokens = [
        (token, score) for token, score in zip(merged_tokens, merged_scores) if score > threshold
    ]

    # Fallback: top 2 if none pass
    if not important_tokens:
        sorted_tokens = sorted(
            [(token, score) for token, score in zip(merged_tokens, merged_scores) if score > 0],
            key=lambda x: x[1],
            reverse=True
        )
        important_tokens = sorted_tokens[:2]

    return important_tokens, threshold

# Run and print results
for text in test_sentences:
    important_tokens, threshold = get_important_tokens(text)
    print(f" Text: {text}")
    print(f"Important Tokens: {important_tokens}")
    print(f"Threshold: {round(threshold, 4)}")



📝 Text: Insulin can reverse diabetes permanently.
📌 Important Tokens: [('diabetes', 0.07572530955076218)]
📊 Threshold: 0.0707

📝 Text: Drinking bitter leaf tea cures diabetes overnight.
📌 Important Tokens: [('cures', 0.3244030438363552), ('diabetes', 0.34785395860671997)]
📊 Threshold: 0.2373

📝 Text: Regular exercise helps manage blood sugar levels effectively.
📌 Important Tokens: [('blood', 0.8433975577354431)]
📊 Threshold: 0.3819

📝 Text: Using herbal supplements alone can replace diabetes medication.
📌 Important Tokens: [('diabetes', 0.2604775130748749)]
📊 Threshold: 0.2605

📝 Text: Metformin improves insulin sensitivity but is not a permanent solution.
📌 Important Tokens: [('.', 0.06704144924879074)]
📊 Threshold: 0.0536
